In [86]:
import os
import traceback
import pickle
from pandas import DataFrame as df
import pandas as pd

import gradio as gr

import classes.BMICalculator as bmic
import classes.Encoders as enc
from config import CONFIG

# get child bme ref table
child_bmi_data = CONFIG["CHILD_BMI"]
child_bmi_df = pd.read_csv(child_bmi_data)

# import model
with open ("model.txt", "rb") as file:
    model = pickle.load(file)
    file.close()

def calc_bmi(data):
        height = data[h_text]
        height_unit = data[h_unit][0]
        weight = data[w_text]
        weight_unit = data[w_unit][0]
    
        if height_unit == "m":
            imp_obj = bmic.MetricBMICalculator(weight_kg = weight, height_m = height)
            bmi = imp_obj.calculate()
        else:
            imp_obj = bmic.ImperialBMICalculator(weight_lb = weight, height_in = height)
            bmi = imp_obj.calculate()
        
        return bmi

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def request_bmi(data):
    """Validate input, request bmi"""

    try:
        run_status = "Successful run!"
        valid_input = validate_input(data)
        
        if not valid_input:
            raise Exception(run_status)

        bmi = calc_bmi(data)
        return bmi , run_status
    
    except Exception as error:
        traceback.print_exc()
        print (str(error))
        return 0, str(error)

def validate_input(data):
        """ 
        Validate that one unit type was selected and that each is a number.
        """
        
        height = data[h_text]
        weight = data[w_text]
        height_unit = data[h_unit]
        weight_unit = data[w_unit]

        is_zero = any([i == 0 for i in [height, weight]])
        missing_data = any([i is None for i in [height, weight, height_unit, weight_unit]]) 
        if missing_data or is_zero:
            run_status = "Please enter information and try again."
            raise Exception(run_status)
        
        # check if units are consistent
        imperial = ["in", "lb"]
        metric = ["m", "kg"]
        is_imperial = set(imperial) == set([height_unit, weight_unit])
        is_metric = set(metric) == set([height_unit, weight_unit])
        consistent = any([is_imperial, is_metric])
        
        if not consistent:
             run_status = "BMI input units need to be consitently metric (kg and m) or imperial (in and lb). Please try again"
             raise Exception(run_status)
        
        # check that each only contains numbers
        valid_input = all(isfloat(i) for i in [height,weight])

        if not valid_input:
            run_status = "All inputs must be numeric"
            raise Exception(run_status)

        return valid_input

def encode_pred(df, final_cols):

    if all(df["age"] > 20):
        bmi = enc.AdultBMIEncoder(df).encode()
    else:
        bmi = enc.ChildBMIEncoder(bmi_table= child_bmi_data, data_table= df).encode()
    enc.one_hot_enc_bmi(df,"bmi_buckets")
    enc.GenderEncoder(df,"gender").encode()
    enc.AgeEncoder(df,"age").encode()
    enc.MarriageEncoder(df,"ever_married").encode()
    enc.ResidenceEncoder(df,"Residence_type").encode()
    enc.SmokingStatusEncoder(df, "smoking_status").encode()
    enc.HypertensionEncoder(df,"hypertension").encode()
    enc.HeartDiseaseEncoder(df,"heart_disease").encode()
    df = df[final_cols]

    return df

def prep_pred_data(data):
    """
    Create dataframe to make a prediction on.
    
    """
    new_df_dict = {}

    new_df_dict["bmi"] = data[bmi]
    new_df_dict["gender"] = data[gend_in]
    new_df_dict["age"] = data[age_in]
    new_df_dict["ever_married"] = data[marr_in]
    new_df_dict["Residence_type"] = data[res_in]
    new_df_dict["smoking_status"] = data[smoke_in]
    new_df_dict["hypertension"] = data[hyp_in]
    new_df_dict["heart_disease"] = data[hdis_in]
    
    col_order = ["age", "hypertension",	"heart_disease", "bmi_under",	"bmi_healthy",	"bmi_over",	"bmi_obese", "is_male",	"c_ever_married",	"live_urban",	"ss_1",	"ss_2",	"ss_3",	"ss_4"]
    pred_df = df.from_dict([new_df_dict])
    encoded_pred_df = encode_pred(pred_df, col_order)

    return encoded_pred_df

def request_prediction(data):
    """
    Request user input and predict data
    
    """
    try:
        pred_df = prep_pred_data(data)
        stroke_pred = model.predict(pred_df)
        stroke = "Stroke predicted"
        no_stroke = "No stroke predicted"
        pred_val = stroke if stroke_pred == 1 else no_stroke
        return pred_val
    except Exception as error:
        traceback.print_exc()
        print (str(error))

    


### Demo

In [91]:
#demo


with gr.Blocks() as demo:
    gr.Markdown(""" 
                    # Stroke Predictor  
                    This model takes user submitted information and makes a prediction of whether that information is predicted to
                    result in a stroke. [A stroke, sometimes called a brain attack, occurs when something blocks blood supply to 
                    part of the brain or when a blood vessel in the brain bursts. 
                    In either case, parts of the brain become damaged or die.](https://www.cdc.gov/stroke/about.htm#:~:text=A%20stroke%2C%20sometimes%20called%20a,brain%20become%20damaged%20or%20die.)

                """)
    gr.Markdown("""
                        *Disclaimer: This demo is for educational purposes only. I'm not a doctor and this is not intended to
                        inform any medical decisions.*
                        """)
    # calc bmi logic 
    with gr.Row():
        with gr.Column():
            gr.Markdown("""
                            ## Inputs
                            Please input your information below.
                            """)
            with gr.Row():
            
                h_text = gr.Number(label = "Height")
                h_unit = gr.Radio(["in", "m"], label = "Height units")
                w_text = gr.Number(label = "Weight")
                w_unit = gr.Radio(["lb", "kg"], label = "Weight units")

            with gr.Row():
                add_btn = gr.Button(value = "Submit")

            with gr.Row(): 
                bmi = gr.Number(label = "BMI", interactive = False)
                run_status = gr.Text(label= "Run Status")
            
            add_btn.click(request_bmi, inputs={h_text, h_unit, w_text, w_unit}, outputs={run_status, bmi})
            
            # get the rest of the inputs and input bmi
            with gr.Row():
                age_in = gr.Number(label = "Age")
                gend_in = gr.Radio(choices = ["Male", "Female"], label="Gender?")
                marr_in = gr.Radio(choices = ["Yes", "No"], label="Ever Married?")

            with gr.Row():
                res_in = gr.Radio(choices = ["Urban", "Rural"], label="Residence Type")
                hyp_in = gr.Radio(choices = ["Yes", "No"], label="Hypertension?")
                hdis_in = gr.Radio(choices = ["Yes", "No"], label="Heart Disease?")

            with gr.Row():    
                smoke_in = gr.Radio(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)
                
            with gr.Row():
                submit_btn = gr.Button("Submit")
            

        with gr.Column():
            with gr.Row():
                gr.Markdown("""
                            ## Prediction
                            After information submission, your prediction will appear here.
                            """)
            pred = gr.Textbox(value = "...", label = "Prediction")
            disclaimer = gr.PlayableVideo(value = CONFIG["NOT_DOC"])
            submit_btn.click(request_prediction, inputs = {age_in, gend_in, res_in, hyp_in, hdis_in, smoke_in, marr_in, bmi}, outputs= pred)
    # bmi.change()
    # submit_btn.click(request_prediction, inputs={gend_in}, outputs= None)
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.
